# Telecom X - Análise de Evasão de Clientes

Você foi contratado como assistente de análise de dados na Telecom X e 
fará parte do projeto "Churn de Clientes". A empresa enfrenta um alto 
índice de cancelamentos e precisa entender os fatores que levam à perda 
de clientes.

Seu desafio será coletar, tratar e analisar os dados, utilizando Python e
 suas principais bibliotecas para extrair insights valiosos. A partir da
 sua análise, os demais colegas da  equipe de Data Science poderá 
avançar para modelos preditivos e desenvolver estratégias para reduzir a
 evasão.

# 📌 Extração

Nesta etapa, importamos as bibliotecas necessárias e carregamos os dados do arquivo JSON. Utilizamos a função `json_normalize` do Pandas para transformar a estrutura aninhada do JSON em um DataFrame plano, facilitando a manipulação.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Carregando o arquivo JSON
data = pd.read_json('TelecomX_Data.json')

# Normalizando o JSON para um formato tabular (DataFrame)
df_raw = pd.json_normalize(data.to_dict('records'))

# Exibindo as primeiras 5 linhas para verificação
print("Dados brutos carregados:")
display(df_raw.head())

# Exibindo informações gerais sobre o DataFrame
print("\nInformações do DataFrame original:")
df_raw.info()

# 🔧 Transformação

Aqui, realizamos a limpeza e a preparação dos dados para a análise. Isso inclui:
1. **Remoção de Dados Faltantes:** Linhas com a coluna `Churn` vazia foram removidas.
2. **Correção de Tipos de Dados:** A coluna de gastos totais (`account.Charges.Total`) foi convertida para um formato numérico, e os valores ausentes (que surgiram de espaços em branco) foram preenchidos com a mediana.
3. **Renomeação de Colunas:** Os nomes das colunas foram simplificados para facilitar o acesso e a leitura do código.

In [ ]:
# Filtrando linhas onde a coluna 'Churn' não está vazia
df = df_raw[df_raw['Churn'] != ''].copy()

# A coluna 'account.Charges.Total' tem espaços em branco que precisam ser tratados.
# 'coerce' transforma os valores inválidos (espaços) em NaT (Not a Time) / NaN (Not a Number)
df['account.Charges.Total'] = pd.to_numeric(df['account.Charges.Total'], errors='coerce')

# Preenchendo os valores nulos (NaN) resultantes com a mediana da coluna
median_total_charges = df['account.Charges.Total'].median()
df['account.Charges.Total'].fillna(median_total_charges, inplace=True)

# Renomeando as colunas para nomes mais simples e em português
new_column_names = {
    'customerID': 'ID_Cliente',
    'Churn': 'Churn',
    'customer.gender': 'Genero',
    'customer.SeniorCitizen': 'Idoso',
    'customer.Partner': 'Parceiro',
    'customer.Dependents': 'Dependentes',
    'customer.tenure': 'Meses_Contrato',
    'phone.PhoneService': 'Servico_Telefonico',
    'phone.MultipleLines': 'Multiplas_Linhas',
    'internet.InternetService': 'Servico_Internet',
    'internet.OnlineSecurity': 'Seguranca_Online',
    'internet.OnlineBackup': 'Backup_Online',
    'internet.DeviceProtection': 'Protecao_Dispositivo',
    'internet.TechSupport': 'Suporte_Tecnico',
    'internet.StreamingTV': 'Streaming_TV',
    'internet.StreamingMovies': 'Streaming_Filmes',
    'account.Contract': 'Contrato',
    'account.PaperlessBilling': 'Fatura_Online',
    'account.PaymentMethod': 'Metodo_Pagamento',
    'account.Charges.Monthly': 'Gasto_Mensal',
    'account.Charges.Total': 'Gasto_Total'
}
df.rename(columns=new_column_names, inplace=True)

# Exibindo informações do DataFrame após a transformação
print("Informações do DataFrame transformado:")
df.info()

# Verificando se ainda há valores nulos
print("\nVerificação de valores nulos após tratamento:")
print(df.isnull().sum())

# 📊 Carga e Análise Exploratória (EDA)

Com os dados limpos e prontos, iniciamos a análise exploratória para extrair insights. Criamos diversas visualizações para entender:
- A proporção geral de evasão de clientes.
- A relação entre o churn e características demográficas (gênero, idade, parceiros, dependentes).
- O impacto do tipo de contrato na retenção de clientes.
- A distribuição de gastos mensais e totais entre clientes que evadiram e os que permaneceram.

In [ ]:
# Configurando o estilo dos gráficos
sns.set_style('whitegrid')

# 1. Visualização da distribuição do Churn
plt.figure(figsize=(7, 7))
df['Churn'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'salmon'], explode=(0, 0.05))
plt.title('Distribuição de Evasão de Clientes (Churn)', fontsize=16)
plt.ylabel('')
plt.show()

# 2. Evasão por Tipo de Contrato
plt.figure(figsize=(10, 6))
sns.countplot(x='Contrato', hue='Churn', data=df, palette='viridis')
plt.title('Evasão por Tipo de Contrato', fontsize=16)
plt.xlabel('Tipo de Contrato', fontsize=12)
plt.ylabel('Contagem de Clientes', fontsize=12)
plt.show()

# 3. Evasão por Cliente ser Idoso ou não
plt.figure(figsize=(8, 6))
sns.countplot(x='Idoso', hue='Churn', data=df, palette='pastel')
plt.title('Evasão por Cliente Idoso', fontsize=16)
plt.xlabel('Cliente é Idoso?', fontsize=12)
plt.ylabel('Contagem de Clientes', fontsize=12)
plt.xticks([0, 1], ['Não', 'Sim'])
plt.show()

# 4. Relação entre Parceiro, Dependentes e Churn
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
sns.countplot(ax=axes[0], x='Parceiro', hue='Churn', data=df, palette='coolwarm')
axes[0].set_title('Evasão por Clientes com Parceiro(a)')
axes[0].set_xlabel('Possui Parceiro(a)')
axes[0].set_ylabel('Contagem de Clientes')

sns.countplot(ax=axes[1], x='Dependentes', hue='Churn', data=df, palette='coolwarm')
axes[1].set_title('Evasão por Clientes com Dependentes')
axes[1].set_xlabel('Possui Dependentes')
axes[1].set_ylabel('')
plt.suptitle('Análise de Evasão por Relacionamento', fontsize=18)
plt.show()

# 5. Distribuição dos Gastos Mensais por Churn
plt.figure(figsize=(12, 7))
sns.histplot(data=df, x='Gasto_Mensal', hue='Churn', multiple='stack', kde=True, palette='magma')
plt.title('Distribuição dos Gastos Mensais por Churn', fontsize=16)
plt.xlabel('Gasto Mensal', fontsize=12)
plt.ylabel('Densidade de Clientes', fontsize=12)
plt.show()

# 💡 Insights e Conclusões

A partir da análise exploratória, podemos extrair os seguintes insights valiosos para a Telecom X:

1.  **Taxa de Evasão Geral:** Cerca de **26.6%** da base de clientes analisada cancelou o serviço. Este é um número significativo que justifica a investigação.

2.  **Tipo de Contrato é Crucial:** O gráfico de "Evasão por Tipo de Contrato" mostra claramente que clientes com **contratos mensais (`Month-to-month`) têm uma taxa de evasão drasticamente maior** em comparação com aqueles com contratos de 1 ou 2 anos. Isso sugere que clientes com vínculos mais longos são mais fiéis. 
    * **Ação Sugerida:** Criar campanhas de incentivo para que clientes de contratos mensais migrem para planos anuais ou bianuais, oferecendo descontos ou benefícios.

3.  **Clientes Idosos Evadem Mais:** A proporção de clientes idosos que cancelam o serviço é visivelmente maior do que a de não idosos. 
    * **Ação Sugerida:** Investigar se os serviços e o suporte oferecidos estão adequados para este público. Talvez seja necessário um atendimento mais simplificado ou pacotes de serviços mais direcionados.

4.  **Clientes sem Vínculos Familiares Tendem a Sair:** Clientes que não possuem parceiro(a) e não possuem dependentes apresentam uma maior propensão à evasão. 
    * **Ação Sugerida:** Desenvolver ofertas voltadas para famílias ou casais, como pacotes com múltiplas linhas ou descontos progressivos, para aumentar a retenção desse grupo.

5.  **Gastos Mensais Elevados e Churn:** O histograma de gastos mensais indica que a evasão é mais concentrada na faixa de clientes que pagam valores mais altos (aproximadamente entre $70 e $100 por mês). 
    * **Ação Sugerida:** Analisar se esses clientes sentem que o custo-benefício está adequado. Pode ser uma oportunidade para oferecer reajustes de plano ou benefícios adicionais para justificar o valor pago.

Estes insights servem como um excelente ponto de partida para a equipe de Data Science aprofundar a investigação e construir modelos preditivos de churn.